# RNN to generate Shakespeare text

The point is to create a RNN model to generate text, this notebook is based on the tensorflow tutorial, https://www.tensorflow.org/tutorials/sequences/text_generation.

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

### Read the datas

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 characters in text
print(text[:250])

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

1122304/1115394 [==============================] - 0s 0us/step
Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

65 unique characters


### Vectorise the text

In [3]:
# Creating a mapping from unique characters to indices
char2idx = {char : index for index, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

{
  'o' :  53,
  'U' :  33,
  '3' :   9,
  'r' :  56,
  's' :  57,
  'Z' :  38,
  'n' :  52,
  'm' :  51,
  'e' :  43,
  'w' :  61,
  'k' :  49,
  'f' :  44,
  'R' :  30,
  '&' :   4,
  'a' :  39,
  'c' :  41,
  'b' :  40,
  'X' :  36,
  'g' :  45,
  'V' :  34,
  ...
}
'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Create training examples and targets

Now that we can represent the text as integers we have to generate training datas for the RNN to learn from.

In [4]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# Creates a Dataset whose elements are slices of the given tensors.

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

# Splits the dataset in batches of size seq_length
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

F
i
r
s
t
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [5]:
# Generate the input and target texts
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print ('Input data: ' + repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:' + repr(''.join(idx2char[target_example.numpy()])))

Input data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [6]:
# Goes through the 5 first steps of the first sequence
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Create training batches

In [7]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build the model

In [8]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [9]:
import functools
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True), 
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(vocab), 
    embedding_dim=embedding_dim, 
    rnn_units=rnn_units, 
    batch_size=BATCH_SIZE)

### Built model:

<img src="https://www.tensorflow.org/tutorials/sequences/images/text_generation_training.png" alt="image" width="400"/>

In [10]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()

(64, 100, 65) # (batch_size, sequence_length, vocab_size)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Decode the prediction of the model
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print('Input: \n' + repr(''.join(idx2char[input_example_batch[0]])))
print('')
print('Next Char Predictions: \n' + repr("".join(idx2char[sampled_indices ])))

Input: 
' IV:\nNow, brother Richard, will you stand by us?\n\nGLOUCESTER:\nAy, in despite of all that shall withs'

Next Char Predictions: 
"bBPGHt,koCHAJd$ri,kiAfPoHDJahN??-p:\nfhEZDUgCvN3aQ$U'!3 'uRO,hoTCdKn3tI$vCfWftC;:uUtX,k,rlxZ;XXUQXW?F"


## Train the model

In [12]:
def loss(labels, logits):
  return tf.keras.backend.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: " + str(example_batch_predictions.shape) + " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      " + str(example_batch_loss.numpy().mean()))

Prediction shape: (64, 100, 65) # (batch_size, sequence_length, vocab_size)
scalar_loss:      4.174783


In [13]:
model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [15]:
EPOCHS=3

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
174/174 [==============================] - 837s 5s/step - loss: 2.6824
Epoch 2/3
174/174 [==============================] - 824s 5s/step - loss: 1.9257
Epoch 3/3
174/174 [==============================] - 820s 5s/step - loss: 1.6746


In [17]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [18]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u'ROMEO: '))

ROMEO: that of used you that make doint
Hastime him loved as pery sevol them bray?

Second, you he not, and hear is
kneeth our fawht of Glauporem, come.

MENENIUS:
He's not by when at with the dost shoued her rayer?

Nusse:
Heful being downt we have surfed, for thy vers of you
That you stain: the bressand stand of wife's dray:
The mins of a myed,
and we him boings, but he hasty, betis brown himsell Withap wroth,
We put tume. Kim!

KING LEVHARD:
Clawess of the grysty sovereigeng; herce, that you goosess of fimon my grave
Bemance: where ill and the proplets; sir manithards, duty porroms
Nath us Cart alliouble crueson: who would, I'll each
shoolf the spin our appetted dy?

RUCENTOR:
Well he hadding in my bring.

TAUMENTIO:
Proy is the water been to piry no faithering.

TRANCO:
Shall be gone to thou, intence; but those thise fairth,
Long Edfin, like vair against thy fuel may, on't you lizent?

WrTWithart, with him, my insure.

WARWICK:
Whice you mu, eeters, not you are Potingmen death,
I l